# <span style="color:purple; font-weight:bold">GridSearch Cross Validation</span>      

GridSearchCV is a technique used for HyperParameters tuning.                

## <span style="color:blue; font-wieght:bold">Hyper Parameter</span> 

- A hyperparameter is an external configuration variable set before the training process begins that controls how the model learns.                   
- The "hyper" prefix indicates that it's a "top-level" variable that governs the learning process itself, unlike model parameters (like weights and biases) which are internal variables learned from the data during training.                 

![alt text](assets/images/HyperParameters_Types.png)

![alt text](assets/images/Hyper_Parameter_Tuning.png)                           

         

---         
## There are three popular tehcniques used for finding optimal values of hyper parameters.                     


### <span style="color:blue; font-wieght:bold">1. GridSearch Cross Validation                              
### <span style="color:blue; font-wieght:bold">2. RandomizedSearch Cross Validation                             
### <span style="color:blue; font-wieght:bold">3. Bayesian Optimization                                 

![alt text](assets/images/Grid_Vs_Randomized_Vs_Bayesian.png)
 

 ---

While GridSearchCV and RandomizedSearch CV use basics maths (looping, statistics, average, probability distribution), Bayesian Optimization involves advanced maths.                
Bayesian Optimization = Use Gaussian Processes to model performance + Acquisition Function to pick best next point based on mean + uncertainty.                  

For Bayesian Optimization we should only:                

✔ Understand what Bayesian Optimization does               
✔ Know when to use it (vs Grid/Random)               
✔ Learn how to set search spaces                   
✔ Know how acquisition functions behave                
✔ Use libraries like sklearn-optuna-hyperopt                   


If still interested, Best Resources to understand the maths behind it:                          
 https://distill.pub/2020/bayesian-optimization/                    

 ----

## <span style="color:purple"> Let's now use different models with different hyperparameters using GridSearchCV api and try finding best hyperparameters values for these models, all in one single line.

In [78]:
# Loading iris flower dataset from sklearn
from sklearn import svm, datasets
iris = datasets.load_iris()

In [63]:
import pandas as pd
df = pd.DataFrame(iris.data, columns=iris.feature_names)

# Add a new 'flower' column containing the numerical target values (0, 1, 2)
df['flower'] = iris.target
# Map the numerical targets to actual flower names (e.g., 'setosa', 'versicolor') using a lambda function
df['flower'] = df['flower'].apply(lambda x:iris.target_names[x])
df[47:150]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


## <span style="color:blue; font-weight:bold"> 1. Lets try and optimise hyperparameters manually using Train-Test Split method.</span>

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [65]:
model = svm.SVC(kernel='rbf', C=30, gamma='auto')
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9333333333333333

## <span style="color:blue; font-weight:bold"> 1. Let's now  use K-Fold Cross Validation to optimise parameters of the model.</span>                    

**I am manually supplying different parameters value to the model using sklearn's cross_val_score() method with 5-fold cross validation.**


In [66]:
from sklearn.model_selection import cross_val_score

cross_val_score(svm.SVC(kernel='linear', C=10, gamma='auto'), iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [67]:
cross_val_score(svm.SVC(kernel='rbf', C=10, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [68]:
cross_val_score(svm.SVC(kernel='rbf', C=20, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

**The above steps used by me is manual and not efficient.**            

**I am using for loop now to achieve same thing**       

In [69]:
import numpy as np
kernels = ['rbf', 'linear']
C = [1, 10, 20]

avg_scores = {}

for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval, C=cval, gamma='auto'), iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)
        
avg_scores       

{'rbf_1': np.float64(0.9800000000000001),
 'rbf_10': np.float64(0.9800000000000001),
 'rbf_20': np.float64(0.9666666666666668),
 'linear_1': np.float64(0.9800000000000001),
 'linear_10': np.float64(0.9733333333333334),
 'linear_20': np.float64(0.9666666666666666)}

**These results tell me that optimal values for Hyperparameters in SVC for this dataset are:**              

1. kernel = 'rbf'                   
2. C = '1'                  
3. gamma = 'auto'                   

**We can use GridSearchCV api which does exact same thing as the above for loop code but in a single line**

In [70]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1, 10, 20],
    'kernel': ['rbf', 'linear']
}, cv=5, return_train_score=False)

clf.fit(iris.data, iris.target)
clf.cv_results_


{'mean_fit_time': array([0.00075102, 0.00057797, 0.0006083 , 0.00061078, 0.00068803,
        0.00061669]),
 'std_fit_time': array([4.17340129e-05, 5.96001433e-05, 2.90348804e-05, 4.99868645e-05,
        9.73563694e-05, 5.24392112e-05]),
 'mean_score_time': array([0.00052118, 0.00039501, 0.00040722, 0.00041952, 0.00047827,
        0.00038557]),
 'std_score_time': array([8.61486532e-05, 1.75961311e-05, 1.02303520e-05, 6.06829570e-05,
        3.97897680e-05, 1.81451420e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
 

**I know that's not highly readable. So here's the better view**

In [71]:
df = pd.DataFrame(clf.cv_results_)
df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [72]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [73]:
clf.best_score_

np.float64(0.9800000000000001)

In [74]:
#dir(clf)           #You can check all attritbutes to play around

## We can also use <ins>[RandomisedSearchCV](https://blog.dailydoseofds.com/p/grid-search-vs-random-search-vs-bayesian)</ins> where we don't specify range of values for the parameters.                    

**The api tries different combination of parameters.**                                      

**This reduces number of iterations and is useful when there are large number of parameters in the model and our computation resources are limited.**

In [75]:
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
            'C': [1, 10, 20],
            'kernel': ['rbf', 'linear']
}, cv=5, return_train_score=False, n_iter=2)

rs.fit(iris.data, iris.target)

df = pd.DataFrame(rs.cv_results_)

df[['param_C', 'param_kernel', 'mean_test_score']]  # Let's see what parameters values are giving higher accuracy for the model using RandomisedSearchCV api


,param_C,param_kernel,mean_test_score
0,10,linear,0.973333
1,1,linear,0.980000


In [76]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='lbfgs', max_iter=1000),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [77]:
scores = []

for model_name, mp in model_params.items():
        clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
        clf.fit(iris.data, iris.target)
        scores.append({
            'model': model_name,
            'best_score': clf.best_score_,
            'best_params': clf.best_params_
        })

df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df       

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.953333,{'n_estimators': 10}
2,logistic_regression,0.973333,{'C': 1}


**Based on the above results of my GridSearchCV on different models i can conclude that SVM model gives me highest accuracy with <e style=color:blue>'C'=1</e> and <e style=color:blue>kernel='rbf'</e>**